<a href="https://colab.research.google.com/github/giorgioiacono001-bit/Financial-Markets-Radar/blob/main/momentum_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

df = pd.read_excel(file_name)

Saving prezzi_rendimenti.xlsx to prezzi_rendimenti.xlsx


In [ ]:
#La velocità è il rendimento a 20 giorni e l'accelerazione è il rapporto tra rendimenti 5 giorni e rendimento 20 giorni
df["velocity"] = df["ret_20d"]

df["acceleration"] = np.where(
    df["ret_20d"] != 0,
    df["ret_5d"] / df["ret_20d"],
    np.nan
)

#Persistenza, ossia qualità del movimento: quanti giorni sono positivi negli ultimi N?
df["pos_day"] = (df["ret_1d"] > 0).astype(int)

df["persistence_20d"] = (
    df.groupby("ticker")["pos_day"]
    .rolling(20)
    .sum()
    .reset_index(level=0, drop=True)
)

#Trend: prezzo sopra SMA50 + direzione positiva: se 1 momentum a favore di trend; se 0 controtrend
df["sma_50"] = (
    df.groupby("ticker")["adj_close"]
    .rolling(50)
    .mean()
    .reset_index(level=0, drop=True)
)

df["trend_flag"] = np.where(
    (df["adj_close"] > df["sma_50"]) &
    (df["sma_50"] > df.groupby("ticker")["sma_50"].shift(5)),
    1,
    0
)


#Anomalia: quanto il rendimento attuale è fuoriscala rispetto al titolo stesso: Z>2 movimento anomalo; Z>3 evento raro
rolling_mean = (
    df.groupby("ticker")["ret_1d"]
    .rolling(60)
    .mean()
    .reset_index(level=0, drop=True)
)

rolling_std = (
    df.groupby("ticker")["ret_1d"]
    .rolling(60)
    .std()
    .reset_index(level=0, drop=True)
)

df["z_score"] = (df["ret_1d"] - rolling_mean) / rolling_std


#Volumi: volume relativo rispetto alla media dei volumi: <1 scarso interesse; 2 partecipazione forte
df["vol_avg_20"] = (
    df.groupby("ticker")["volume"]
    .rolling(20)
    .mean()
    .reset_index(level=0, drop=True)
)

df["rel_volume"] = df["volume"] / df["vol_avg_20"]

#Momentum relativo: non abbiamo ticker FTSEMIB quindi assumiamo come rendimento di mercato la media dei rendimenti
# rendimento medio di mercato (20 giorni) per ogni data
market_ret_20d = (
    df.groupby("date")["ret_20d"]
    .median()   # usa median, più robusta della mean
    .rename("market_ret_20d")
)

df = df.merge(market_ret_20d, on="date", how="left")

# momentum relativo
df["relative_momentum"] = df["ret_20d"] - df["market_ret_20d"]

#se 0 titolo sovraperforma; se <0 sottoperforma


In [ ]:
#NORMALIZZAZIONE
def zscore_cross(x):
    return (x - x.mean()) / x.std()

features = [
    "velocity",
    "acceleration",
    "persistence_20d",
    "z_score",
    "rel_volume",
    "relative_momentum"
]

for f in features:
    df[f + "_z"] = df.groupby("date")[f].transform(zscore_cross)


In [ ]:
#Valore momentum
df["momentum_score"] = (
    0.25 * df["velocity_z"] +
    0.20 * df["acceleration_z"] +
    0.15 * df["persistence_20d_z"] +
    0.15 * df["z_score_z"] +
    0.15 * df["rel_volume_z"] +
    0.10 * df["relative_momentum_z"]
)


In [ ]:
# @title
#Top titoli da guardare oggi
latest_date = df["date"].max()

watchlist = (
    df[df["date"] == latest_date]
    .sort_values("momentum_score", ascending=False)
    .head(30)
)

watchlist[[
    "ticker",
    "momentum_score",
    "velocity",
    "acceleration",
    "persistence_20d",
    "z_score",
    "rel_volume",
    "relative_momentum"
]]


,ticker,momentum_score,velocity,acceleration,persistence_20d,z_score,rel_volume,relative_momentum
26131,LUVE.MI,1.253524,-0.013836,1.797831,11.0,-3.361447,14.973013,-0.004834
35914,SPM.MI,1.248600,0.221963,0.042024,17.0,0.210895,0.892150,0.230965
1111,ADB.MI,1.089522,0.101449,0.547619,9.0,2.195559,4.402393,0.110452
32300,PRY.MI,1.069582,0.179537,0.138623,13.0,1.524707,0.817235,0.188540
21683,IEG.MI,1.060925,0.232798,0.723710,13.0,0.195182,0.481920,0.241801
37860,TEN.MI,0.930664,0.144100,0.358819,14.0,0.764387,0.875394,0.153102
2501,ANIM.MI,0.917476,0.138072,0.447143,15.0,-0.076249,1.446888,0.147074
26409,MAIRE.MI,0.877515,0.141159,0.318067,14.0,0.640823,0.642359,0.150161
39528,TIT.MI,0.862242,0.113797,0.447088,13.0,1.170202,1.083571,0.122799
16401,ENI.MI,0.852003,0.110295,0.322115,12.0,1.476986,1.310400,0.119298


In [ ]:
df["momentum_change"] = df.groupby("ticker")["momentum_score"].diff(5)

In [ ]:
df["volatility_20d"] = (
    df.groupby("ticker")["ret_1d"]
    .rolling(20)
    .std()
    .reset_index(level=0, drop=True)
)

df["volatility_20d_z"] =(
    df.groupby("date")["volatility_20d"]
    .transform(zscore_cross)
)

In [ ]:
df["combined_index"] = (
    0.6 * df["momentum_score"] +
    0.4 * df["momentum_change"]
)

In [ ]:
#Salvataggio
output_file = "momentum_surveillance_system.xlsx"
df.to_excel(output_file, index=False)

files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#FIGURA
fig = px.scatter(
    df_last,
    x="momentum_score",
    y="volatility_20d_z",
    hover_name="ticker",
    title=f"Momentum vs Volatilità — {last_date}"
)
fig.show()

#FIGURA 1
fig1 = px.scatter(
    df_last,
    x="momentum_score",
    y="momentum_change",
    hover_name="ticker",
    title=f"Momentum vs Momentum Change — {last_date}"
)
fig1.show()

#FIGURA 2
df["momentum_score_5d_avg"] = (
    df.groupby("ticker")["momentum_score"]
    .rolling(5)
    .mean()
    .reset_index(level=0, drop=True)
)

df["momentum_change_5d_avg"] = (
    df.groupby("ticker")["momentum_change"]
    .rolling(5)
    .mean()
    .reset_index(level=0, drop=True)
)

fig2 = px.scatter(
    df_last_avg,
    x="momentum_score_5d_avg",
    y="momentum_change_5d_avg",
    hover_name="ticker",
    title=f"5-day Momentum Score Avg vs. 5-day Momentum Change Avg — {last_date}"
)
fig2.show()
